In [ ]:
#youtube link::::     https://www.youtube.com/watch?v=yat7soj__4w
    
    
# 🔗  Link for world cups 1930-2018: https://en.wikipedia.org/wiki/2014_FI...
# 🔗  Link for world cup 2022 (with data before starting the World Cup) : https://web.archive.org/web/202211150... 
# 👨🏻‍💻  Code/Dataset used in this video: https://github.com/ifrankandrade/fifa...
# 🔗 Link datasets (2nd options): https://drive.google.com/drive/folder...
# 🔗 Step-by-step guide to get the missing data: https://medium.com/geekculture/how-to...

# 🔥 Join My Automation Course in Python: https://www.udemy.com/course/automate...
# 🔥 8-hour Web Scraping Course in Python: https://www.udemy.com/course/web-scra...
# 💰 Make money using your Programming & Data Science skills: https://codelessearnmore.teachable.co...
# ✅  Follow me on Medium: https://frank-andrade.medium.com/

In [1]:
# pip install beautifulsoup4
# https://en.wikipedia.org/wiki/2014_FIFA_World_Cup
# pip install request
# pip install lxml

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# Web scraping
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982, 
         1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018]
print(years)



[1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018]


In [3]:
def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    matches = soup.find_all('div', class_='footballbox')

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())


    # so we create a dictionary for our work 
    dict_football = {'home': home, 'score': score, 'away': away}

    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    return df_football



#fifa_worldcup_history_data

fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)  #the pd.concat is to arrange all the tables in the world cups into a dataframe table 

df_fifa.to_csv('fifa_worldcup_historical_data.csv', index=False)




df_features = get_matches(2022)
df_features.to_csv('fifa_worldcup_features.csv', index=False)



In [5]:
# Cleaning the missing Data
df_historical = pd.read_csv('fifa_worldcup_historical_data.csv')
df_features = pd.read_csv('clean_fifa_worldcup_fixture.csv')
df_missing_data = pd.read_csv('fifa_worldcup_missing_data.csv')


In [10]:
df_features['home'] = df_features['home'].str.strip()   #the string strip is to remove extra spaces
df_features['away'] = df_features['away'].str.strip()

In [14]:
#null data
# df_missing_data[df_missing_data['home'].isnull()]

#drop null data
df_missing_data.dropna(inplace=True)


In [19]:
#concatena dfs and clean
df_final_historical = pd.concat([df_historical, df_missing_data], ignore_index=True)
df_final_historical.drop_duplicates(inplace=True)
df_final_historical.sort_values('year', inplace=True)

df_final_historical

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
863,Brazil,2–0,Costa Rica,2018
864,Serbia,1–2,Switzerland,2018
865,Serbia,0–2,Brazil,2018
867,Germany,0–1,Mexico,2018


In [25]:
#Deleting match with walk-over
delete_index = df_final_historical[df_final_historical['home'].str.contains('Sweden') & 
                                   df_final_historical['away'].str.contains('Austria')].index

df_final_historical.drop(index=delete_index, inplace=True)

In [38]:
#columns scores with not only digits and "-" -->[^]: Matches character not in the bracket

df_final_historical['score'] = df_final_historical['score'].str.replace('[^\d–]', '', regex=True)

In [39]:
#cleaning the home and away column

df_final_historical['home'] = df_final_historical['home'].str.strip()   #the string strip is to remove extra spaces
df_final_historical['away'] = df_final_historical['away'].str.strip()

In [40]:
#splitting scores column into home and away goals and dropping score column
df_final_historical['score'].str.split('–')

0      [4, 1]
17     [4, 2]
16     [6, 1]
15     [6, 1]
14     [1, 0]
        ...  
863    [2, 0]
864    [1, 2]
865    [0, 2]
867    [0, 1]
899    [2, 0]
Name: score, Length: 1344, dtype: object